<a href="https://colab.research.google.com/github/sarvesh237/lfkdsahkhfa/blob/master/NewsRecommenderAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libraries

In [79]:
import pandas as pd
import numpy as np

#for lemmatization
import nltk
nltk.download("wordnet")
nltk.download('stopwords')
#to remove stop words
from nltk.corpus import stopwords

#for tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer

#cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

#gaussian mixture
from sklearn.mixture import GaussianMixture

from sklearn.decomposition import TruncatedSVD# TruncatedSVD


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Importing the collected data**

In [80]:
df_1 = pd.read_csv("https://github.com/sarvesh237/NewsRecommenderIDC401/raw/master/agrima_news_data.csv")
df_2 = pd.read_csv("https://github.com/sarvesh237/NewsRecommenderIDC401/raw/master/sarvesh_news_data.csv")
df_3 = pd.read_csv("https://github.com/sarvesh237/NewsRecommenderIDC401/raw/master/satender_news_data.csv")
df_4 = pd.read_csv("https://github.com/sarvesh237/NewsRecommenderIDC401/raw/master/vishal_news_data.csv")

# **Preprocessing the csv files.**

In [81]:
del df_1['Unnamed: 0']
del df_2['index']
del df_2['Unnamed: 0']
df_2.rename(columns = {'content':'Content'}, inplace = True)
del df_3['Unnamed: 0']
del df_3['Title']
del df_4['Unnamed: 0']

**Remove capitalization, numbers, stop words, non-english characters, drop empty rows and duplicates.**

In [82]:
news_corpus = pd.concat([df_1,df_2,df_3,df_4],ignore_index=True)
news_corpus = news_corpus.dropna() #dropping NaN
news_corpus = news_corpus[news_corpus.Content != ''] #dropping empty rows
news_corpus = news_corpus.drop_duplicates()
news_corpus["Content"] = news_corpus['Content'].str.replace('[^\w\s]',' ')
news_corpus = news_corpus[news_corpus['Content'].map(lambda x: x.isascii())] #remove non-english #find a better way
news_corpus['Content'] = news_corpus['Content'].str.lower() #convert to lowercase
news_corpus.replace('\d+', '', regex=True, inplace=True) #remove numbers

stop_words = stopwords.words('english')
stop_words.extend(['span','class','spacing','href','html','http','title'])
news_corpus['Content'] = news_corpus['Content'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
news_corpus = news_corpus.apply(lambda x: [item for item in x if item not in stop_words])


**Lemmatization**

In [83]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

news_corpus['Content'] = news_corpus.Content.apply(lemmatize_text)
news_corpus['Content'] = news_corpus['Content'].apply(', '.join) #convert list of words to a sentence
news_corpus.replace(',', '', regex=True, inplace=True) # remove commas
news_corpus = news_corpus.reset_index(drop=True)
news_corpus


,Content
0,medium report about swedish manufacturer scani...
1,access covid vaccine cooperation technology cl...
2,after severe criticism over holding consultati...
3,former congress president rahul gandhi thursda...
4,enforcement directorate attached three immovab...
...,...
4589,over mughal gold coin dating back early centur...
4590,china planning spend tibet five year plan allo...
4591,supreme court tuesday came with solution stale...
4592,indian american maju varghese previously serve...


# TF-IDF

In [84]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(news_corpus.Content)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
#denselist = dense.tolist() #very expensive
df_vecs = pd.DataFrame(dense, columns=feature_names)
df_vecs

,aabad,aadarsh,aadat,aadhaar,aadhar,aadhi,aadmi,aage,aajtak,aakash,aaksha,aamir,aamk,aandolan,aandolanjivi,aane,aapada,aapko,aapsu,aaravv,aarogya,aarohan,aarti,aasha,aashay,aashirwad,aasiya,aastha,aasu,aate,aatma,aatmanidbhar,aatmanirbhar,aatmanirbharbharat,aatmanirbharta,aatmiyata,aatmnirbhar,aatmnirbharta,aawam,aaya,...,zhao,zhoigar,zhuoran,ziarul,zila,zilla,zillion,zimbabwe,zindabad,zindagi,zindagii,zinta,ziyad,zojila,zolgensma,zomato,zombie,zonal,zone,zongqi,zoo,zoological,zoology,zoom,zoonotic,zoramthanga,zothankhuma,zoya,zptcs,ztdrktlic,zuali,zubair,zubin,zurbuchen,zurich,zuxxmlt,zwift,zyada,zycov,zydus
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4591,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# **Latent Semantic Analysis : Dimensionality Reduction using TruncatedSVD**

In [85]:
sv_dec = TruncatedSVD(n_components = 50)  # components(no. of latent factors) < n ; n : no. of docs.
ls_analysis = sv_dec.fit_transform(vectors.T)

sv_dec.singular_values_ # eigenvalues for svd (in decreasing order)

array([13.49665211,  8.19753288,  6.65195647,  6.45052751,  6.13464352,
        5.38660275,  5.15440851,  4.58017211,  4.37334406,  4.27804477,
        4.22045317,  4.12363723,  4.06323633,  4.03833941,  3.92994724,
        3.91271931,  3.78116002,  3.73148442,  3.68007841,  3.61918706,
        3.56592061,  3.48169926,  3.40814743,  3.36889205,  3.3115264 ,
        3.29839112,  3.26771276,  3.22544264,  3.14134494,  3.1393752 ,
        3.09795819,  3.06819747,  3.00852638,  2.98422527,  2.95600011,
        2.93716128,  2.91641425,  2.86918444,  2.8406313 ,  2.79317303,
        2.77677357,  2.74052955,  2.73300723,  2.70687799,  2.69635526,
        2.6739246 ,  2.66579904,  2.62971726,  2.61371336,  2.59813829])

In [86]:
variance =  sv_dec.singular_values_/sum(sv_dec.singular_values_) # checking the variance captured by eigenvalues
variance

array([0.06981549, 0.0424042 , 0.03440924, 0.03336729, 0.03173329,
       0.02786382, 0.02666273, 0.02369232, 0.02262244, 0.02212947,
       0.02183156, 0.02133075, 0.02101831, 0.02088952, 0.02032883,
       0.02023972, 0.01955919, 0.01930222, 0.01903631, 0.01872133,
       0.0184458 , 0.01801013, 0.01762967, 0.01742661, 0.01712986,
       0.01706192, 0.01690323, 0.01668457, 0.01624955, 0.01623936,
       0.01602512, 0.01587117, 0.01556251, 0.0154368 , 0.0152908 ,
       0.01519335, 0.01508603, 0.01484172, 0.01469402, 0.01444853,
       0.0143637 , 0.01417621, 0.0141373 , 0.01400214, 0.01394771,
       0.01383168, 0.01378965, 0.013603  , 0.01352022, 0.01343965])

In [87]:
sv_dec = TruncatedSVD(n_components = 25) # components with accepted variance
las = sv_dec.fit_transform(vectors)

#print(las,las.shape)

Topic - Document matrix

In [88]:
# Representation of each doc in terms of latent topics after dimensionality reduction
col = []
for i in range(las.shape[1]):
    col.append("topic_{}".format(i)) 

topic_df = pd.DataFrame(las, columns = col)

topic_df["Docs"] = news_corpus.Content

l = ["Docs"]
for i in col:
    l.append(i)

display(topic_df[l])


,Docs,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,medium report about swedish manufacturer scani...,0.129416,-0.031666,-0.002052,0.016614,-0.058593,-0.036341,-0.017980,0.010633,0.002256,-0.020379,0.012650,0.011991,0.007819,-0.052177,0.008031,-0.015406,0.065748,-0.039627,-0.012163,0.025669,-0.025159,0.021092,-0.005065,-0.046053,-0.052330
1,access covid vaccine cooperation technology cl...,0.241852,-0.033735,0.076399,0.116311,-0.063092,-0.107422,-0.079379,-0.061269,-0.053861,-0.143809,-0.007953,-0.107792,0.029893,0.049869,-0.110122,0.021037,0.123419,0.009363,0.041183,-0.016907,-0.035859,-0.020567,-0.052119,0.070746,0.009397
2,after severe criticism over holding consultati...,0.187095,-0.047153,-0.003890,0.027576,-0.070227,-0.018191,-0.070327,-0.019264,-0.003346,-0.026231,-0.012181,-0.025695,0.021397,-0.005493,0.031371,-0.071900,0.028319,-0.081991,-0.021867,-0.010628,-0.097318,0.075884,-0.117382,-0.192066,-0.175504
3,former congress president rahul gandhi thursda...,0.177060,-0.026036,0.029300,0.010232,-0.039891,-0.040917,-0.059155,0.027508,-0.032294,-0.081084,0.101108,-0.065843,-0.006512,-0.008537,0.022067,0.013118,0.057989,-0.038894,0.002206,0.036223,-0.035020,0.018543,-0.027186,-0.068956,-0.032217
4,enforcement directorate attached three immovab...,0.134256,-0.048095,-0.066803,-0.030220,-0.111663,0.012994,0.045477,0.013996,0.054365,-0.023978,-0.001644,0.026374,-0.039326,-0.090654,0.044467,-0.006812,0.096999,-0.033682,-0.052904,0.018898,0.048245,-0.005248,0.041187,-0.008891,-0.005195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4589,over mughal gold coin dating back early centur...,0.086212,-0.022274,-0.033228,-0.015103,-0.062888,-0.042090,0.091784,0.036596,0.021241,-0.009875,0.003523,0.001519,-0.011665,-0.039641,0.006287,-0.037462,0.001781,-0.012658,0.021049,-0.004338,-0.017384,0.004791,0.003045,0.013742,-0.001952
4590,china planning spend tibet five year plan allo...,0.123348,-0.013563,0.004547,0.000891,-0.026694,-0.072742,-0.033965,-0.033589,-0.019019,-0.025135,-0.066103,-0.000181,0.024848,-0.016887,-0.011579,-0.008367,0.039948,-0.005173,-0.007590,0.007290,-0.020356,-0.030079,0.026949,0.001146,0.018999
4591,supreme court tuesday came with solution stale...,0.178528,-0.066110,-0.074736,-0.003266,-0.120680,0.127771,-0.102600,-0.017891,0.032671,-0.020874,-0.024688,0.011208,0.003979,-0.011992,0.031175,0.009213,0.016957,0.011100,0.016209,-0.003612,-0.034271,0.013393,0.051439,-0.007202,-0.019749
4592,indian american maju varghese previously serve...,0.080712,-0.038111,0.006431,0.013002,-0.024381,-0.039515,-0.024973,0.021028,0.019508,-0.051456,-0.049874,-0.040420,-0.010814,0.058511,-0.027054,-0.059922,0.040850,-0.000925,0.027895,0.043515,-0.012496,-0.010555,0.046021,0.078452,-0.002358


**Visualizing the topics**

In [89]:
'''import umap
import matplotlib.pyplot as plt

embedding = umap.UMAP(n_neighbors=150, min_dist=0.5, random_state=12).fit_transform(las)

plt.figure(figsize=(7,5))
plt.scatter(embedding[:, 0], embedding[:, 1], c = range(4594),s=15 )
plt.show()'''

'import umap\nimport matplotlib.pyplot as plt\n\nembedding = umap.UMAP(n_neighbors=150, min_dist=0.5, random_state=12).fit_transform(las)\n\nplt.figure(figsize=(7,5))\nplt.scatter(embedding[:, 0], embedding[:, 1], c = range(4594),s=15 )\nplt.show()'

**Topics and terms list**

In [90]:
Sigma = sv_dec.singular_values_ 
V_transpose = sv_dec.components_.T
terms = vectorizer.get_feature_names()

for index, component in enumerate(sv_dec.components_):
    zipped = zip(terms, component)
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:5]
    top_terms_list=list(dict(top_terms_key).keys())
    print("Topic "+str(index)+": ",top_terms_list)
del Sigma,V_transpose,terms,zipped,top_terms_key

Topic 0:  ['said', 'that', 'covid', 'case', 'will']
Topic 1:  ['covid', 'vaccine', 'case', 'death', 'vaccination']
Topic 2:  ['election', 'assembly', 'party', 'vaccine', 'congress']
Topic 3:  ['vaccine', 'vaccination', 'farmer', 'dose', 'first']
Topic 4:  ['farmer', 'law', 'farm', 'protest', 'congress']
Topic 5:  ['court', 'vaccine', 'justice', 'supreme', 'vaccination']
Topic 6:  ['police', 'vaccine', 'arrested', 'dose', 'farmer']
Topic 7:  ['sabha', 'price', 'congress', 'rajya', 'budget']
Topic 8:  ['lakh', 'price', 'sabha', 'from', 'fuel']
Topic 9:  ['woman', 'international', 'minister', 'child', 'march']
Topic 10:  ['congress', 'gandhi', 'woman', 'lakh', 'leader']
Topic 11:  ['rawat', 'minister', 'uttarakhand', 'crore', 'singh']
Topic 12:  ['bengal', 'banerjee', 'west', 'mamata', 'price']
Topic 13:  ['sabha', 'army', 'woman', 'kashmir', 'rawat']
Topic 14:  ['congress', 'kashmir', 'jammu', 'worker', 'beneficiary']
Topic 15:  ['price', 'petrol', 'diesel', 'modi', 'fuel']
Topic 16:  ['

# **Creating Users**

In [91]:
#fitting GMM to column "topic_i"

num_cols_topic_df = topic_df.shape[1]-1
gm = []

for i in range(num_cols_topic_df):
  gm.append(GaussianMixture(n_components=10).fit(topic_df.iloc[:, [i]]))


In [92]:
#generating gmm based topic values for each topic for 50 users
#24 x 50 matrix

Users = []

for i in range(50):
  temp = []
  for i in range(num_cols_topic_df):
    temp.append(gm[i].sample(1)[0][0][0])
  Users.append(temp)

Users = pd.DataFrame(Users)
Users.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,0.200653,0.184610,0.231512,0.214012,0.303741,0.159807,0.079170,0.251982,0.147015,0.306544,0.172108,0.181584,0.382944,0.193705,0.041128,0.161326,0.383889,0.141127,0.122109,0.133920,0.258534,0.149594,0.184943,0.209793,0.191621,0.154478,0.238207,0.170291,0.129196,0.152350,0.198615,0.249016,0.096799,0.131567,0.125690,0.133643,0.122285,0.228729,0.176868,0.139311,0.229163,0.145796,0.145438,0.354465,0.173925,0.126957,0.187940,0.110552,0.073976,0.189359
1,-0.034461,0.050158,-0.035508,-0.011466,-0.005813,-0.019417,-0.163518,-0.034329,-0.141363,-0.084978,-0.029129,-0.022692,-0.046263,-0.056362,-0.117939,-0.082248,-0.047706,-0.016775,0.268078,-0.079957,-0.024413,-0.043570,-0.046554,-0.045693,-0.001042,-0.051368,-0.011710,-0.037607,0.001409,-0.164097,-0.046755,-0.054171,-0.071515,-0.034401,0.138154,-0.066518,-0.001906,-0.053370,-0.088089,-0.115363,-0.006210,-0.136521,-0.050628,-0.008647,-0.047825,-0.031556,-0.087683,-0.010625,0.224545,-0.086713
2,0.008502,0.030349,-0.067902,0.045867,0.243496,0.013537,-0.009333,-0.003487,-0.270806,0.277749,0.009728,-0.096583,0.148544,0.158457,-0.009106,-0.016524,-0.061706,0.059742,0.039409,0.358811,0.024182,-0.006188,0.063245,-0.005093,-0.044130,0.002788,-0.095822,-0.094750,-0.173624,-0.103304,-0.130400,-0.073066,0.010115,0.015194,-0.056070,-0.002413,-0.061460,0.049718,-0.003606,0.021194,-0.070622,-0.088092,-0.186585,0.011543,0.017737,0.018031,-0.026699,0.100135,-0.092200,-0.149249
3,0.018163,-0.018709,0.025842,-0.001246,-0.160976,0.235621,0.002945,-0.115483,0.334197,-0.032372,0.150488,0.032570,-0.032474,0.016639,0.014201,-0.019435,0.047076,-0.026559,0.009364,0.062227,-0.100611,0.096809,-0.032227,0.013888,-0.065455,-0.033286,-0.031511,0.004509,0.015720,-0.038977,-0.017128,-0.052406,-0.052764,0.002047,-0.028180,0.038560,-0.043816,-0.000171,0.007527,-0.017813,0.131130,-0.031406,0.003269,0.228984,-0.004935,0.168761,-0.047589,-0.008300,0.121663,-0.002055
4,-0.049865,-0.062031,-0.108032,0.013517,-0.077454,-0.034359,0.055361,-0.080306,-0.044498,-0.102815,-0.058549,-0.082886,-0.086175,-0.000466,-0.038614,-0.059078,-0.149776,0.393313,-0.062841,-0.022093,-0.023103,0.018157,0.085424,-0.039271,0.415435,-0.035660,0.004509,-0.112296,-0.024230,-0.144671,-0.142355,0.014056,0.077299,0.000692,0.135554,-0.090315,0.024645,0.026005,-0.044649,-0.039439,-0.046452,-0.085529,-0.050346,-0.095162,0.136581,0.040610,-0.043532,-0.106816,-0.081064,-0.070343
5,0.092113,-0.073120,-0.044240,-0.092384,-0.099235,-0.043592,-0.012954,0.043190,0.125946,-0.047941,0.065182,-0.041912,0.007311,0.018134,0.029749,-0.031254,-0.005586,-0.054678,0.097219,-0.132970,-0.007174,-0.003521,0.157892,-0.019522,0.097550,0.024877,-0.025455,-0.024850,-0.032709,0.145338,-0.047459,0.101743,-0.074110,-0.027184,-0.066232,-0.066782,-0.099326,0.173128,-0.020972,0.035322,0.100085,-0.010088,-0.095901,-0.024112,-0.025742,0.007987,-0.118882,-0.069260,-0.041518,0.126920
6,0.089707,-0.033295,-0.070672,0.032166,-0.036966,-0.026806,0.082807,-0.136832,0.022599,0.014943,0.150055,-0.051444,0.024062,-0.074572,-0.013506,-0.016921,0.100581,0.082138,-0.097941,0.025519,-0.057113,-0.060999,0.010717,0.137496,-0.055874,-0.023095,-0.030595,0.148743,0.070011,-0.063405,0.004643,-0.000975,0.033522,-0.021273,0.062097,0.118313,-0.009692,-0.038965,0.119070,0.029829,-0.071067,0.014769,-0.037822,0.096697,0.041387,-0.014514,0.036449,-0.020653,-0.092807,0.132280
7,-0.039855,-0.054863,-0.033718,0.110514,-0.011595,0.012093,-0.017252,-0.010482,0.027420,0.008458,-0.078462,0.071128,-0.008178,0.012347,-0.012016,-0.052619,-0.014311,-0.010625,-0.011856,-0.007180,-0.010519,-0.034806,0.008222,-0.058188,0.004839,-0.014829,0.237900,0.060478,-0.008813,-0.025643,-0.004631,-0.003848,-0.000340,0.044351,0.027266,0.006715,0.029783,-0.064860,-0.022141,-0.030328,-0.058011,-0.028370,-0.038356,-0.022746,-0.019199,-0.019314,0.128876,-0.052999,-0.002291,-0.014574
8,0.009382,-0.0211

# **Cosine Similarity between Users and Docs**

In [93]:
#run the above cell before running this twice
slc = list(range(topic_df.shape[1]))
slc.remove(25)
temp_topic_df = topic_df.iloc[:, slc]
Users_temp = Users.T.set_index(temp_topic_df.T.index)

result = temp_topic_df.dot(Users_temp);
result.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4554,4555,4556,4557,4558,4559,4560,4561,4562,4563,4564,4565,4566,4567,4568,4569,4570,4571,4572,4573,4574,4575,4576,4577,4578,4579,4580,4581,4582,4583,4584,4585,4586,4587,4588,4589,4590,4591,4592,4593
0,0.030179,0.049743,0.037136,0.028148,0.048508,0.045585,0.062334,0.022207,0.028918,0.022141,0.021622,0.030890,0.024739,0.017428,0.026682,0.048383,0.032548,0.040429,0.027015,0.038105,0.034933,0.021487,0.044941,0.015186,0.023620,0.021314,0.023164,0.016173,0.045428,0.014759,0.057094,0.043959,0.083301,0.041087,0.036944,0.065385,0.017901,0.024717,0.019972,0.037438,...,0.008717,0.017836,0.014045,0.000163,0.009287,0.009477,0.006085,0.002653,0.009607,0.007297,0.014894,0.008055,0.008048,0.005949,0.001101,0.009921,0.009486,0.011331,0.007112,0.013488,0.012537,0.005971,0.003740,0.012245,0.009304,0.004405,0.011462,0.010232,0.010561,0.009769,0.009081,0.008294,0.028334,0.037309,0.044151,0.022008,0.024645,0.049841,0.013317,0.043782
1,0.036106,0.067152,0.031186,0.048973,0.032816,0.021810,0.046829,0.014434,0.027619,0.050889,0.034297,0.103746,0.030199,0.006486,0.008706,0.017269,0.036291,0.033969,0.018615,0.030526,0.034287,0.039479,0.042439,0.026258,0.000728,0.018751,0.026474,0.010083,0.030137,0.020372,0.020138,0.020619,0.054271,0.004915,0.035049,0.017949,0.018049,0.020930,0.028285,0.026619,...,0.011611,0.016274,0.012399,0.025729,0.032239,0.015274,0.007131,0.008629,0.018310,0.008686,0.025820,0.013256,0.013208,0.010699,0.003668,0.010335,0.026662,0.021555,0.021942,0.017575,0.021745,0.021116,0.005211,0.009774,0.013025,0.016327,0.014223,0.018070,0.009879,0.015923,0.010978,0.011637,0.019239,0.036247,0.032002,0.018436,0.031665,0.028138,0.019187,0.014486
2,0.035917,0.071405,0.057860,0.044257,0.041926,0.057955,0.043953,0.049693,0.027065,0.044791,0.044709,0.098293,0.048912,0.037927,0.011667,0.082793,0.054667,0.070335,0.038847,0.058434,0.061858,0.029348,0.064870,0.039982,0.030524,0.043454,0.042351,0.020686,0.044841,0.021489,0.075099,0.062489,0.052122,0.050834,0.040492,0.066864,0.072187,0.034124,0.041052,0.019779,...,0.006252,0.031522,0.009170,-0.018917,0.020415,0.017493,0.005407,0.008596,0.016071,0.011556,0.021562,0.010608,0.014447,0.005941,0.002806,0.011184,0.016596,0.021252,0.015770,0.010637,0.025324,0.020273,0.003747,0.012729,0.010458,0.013743,0.010998,0.012121,0.008851,0.012574,0.011836,0.008760,0.032764,0.019235,-0.000760,0.026076,0.033054,0.060601,0.013537,0.015517
3,0.040309,0.051928,0.052769,0.057891,0.031161,0.004983,0.039441,0.025716,0.062887,0.038566,0.041279,0.054422,0.026637,0.003044,0.014135,0.001720,0.036900,0.036654,0.021526,0.035930,0.105564,0.026229,0.020763,0.030873,0.058106,0.051961,0.025932,0.012081,0.049088,0.018506,-0.006963,0.016063,0.067771,0.003073,0.046563,0.006296,0.061833,0.026848,0.019449,0.027107,...,0.013338,0.008905,0.016396,0.004641,0.027635,0.014723,0.010067,0.010294,0.018649,0.009249,0.032293,0.014956,0.016122,0.013039,0.003885,0.008552,0.026469,0.025297,0.017012,0.026946,0.022872,0.022175,0.010001,0.014160,0.013861,0.018102,0.018088,0.018942,0.014235,0.017322,0.013737,0.023629,0.018310,0.046358,0.045330,0.028457,0.026269,0.022662,0.016449,0.010844
4,0.032110,0.104227,0.050748,0.067911,0.017698,0.022676,0.130606,0.044263,0.087840,0.055217,0.049325,0.106217,0.040768,0.020248,0.045449,0.074426,0.072888,0.100154,0.041590,0.055378,0.102042,0.036473,0.056745,0.047104,0.047482,0.076712,0.042306,0.026244,0.057745,0.026355,0.032551,0.075504,0.046921,0.031471,0.080608,0.033640,0.106879,0.045563,0.031749,0.018293,...,0.013717,0.018610,0.031749,0.009064,0.039859,0.016521,0.013579,0.009897,0.024717,0.015127,0.022493,0.017667,0.021300,0.013251,0.004345,0.013263,0.033082,0.029464,0.022063,0.035965,0.029033,0.035571,0.008090,0.014356,0.013432,0.020603,0.020737,0.021362,0.015305,0.021258,0.018062,0.029085,0.036337,0.079274,0.087763,0.018882,0.047343,0.031354,0.035831,0.084212
5,0.023038,0.076605,0.038197,0.051694,-0.002853,0.02

# **Find the best 10 documents for each user.**

In [94]:
selected_docs = result.T.apply(lambda s, n: pd.Series(s.nlargest(n).index), axis=1, n=10)
selected_docs.columns =['D1', 'D2', 'D3', 'D4', 'D5','D6','D7','D8','D9','D10']
selected_docs

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10
0,3196,2918,2777,3568,3974,3198,2882,2859,2635,2892
1,1209,544,828,137,222,321,890,871,238,404
2,1688,2124,693,1634,3995,2199,166,90,704,159
3,2269,3474,2295,3762,1891,2491,1708,3402,1794,1852
4,3568,3584,3230,3288,3401,3203,2126,2210,2496,3532
5,3196,2090,2882,3259,2777,3076,3212,2918,3124,2932
6,3991,615,1215,355,3855,368,1575,1159,3340,3353
7,3658,166,1369,157,1688,335,3028,228,2245,2910
8,1028,1387,860,1272,1170,1438,3718,1195,1398,368
9,3568,2210,2496,2137,3288,2126,3584,2414,3230,3401


**Replacing index values by the actual news.**

In [100]:
doc_dict = news_corpus.Content.to_dict()

selected_docs = selected_docs.replace(doc_dict)
selected_docs

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10
0,delhi india march covid vaccination concluded ...,delhi india march delhi lieutenant governor an...,dehradun india march uttarakhand governor baby...,delhi india february total assembly constituen...,delhi india february delhi session court tuesd...,delhi india march cumulative number covid vacc...,delhi india march second phase vaccination cov...,delhi india march ministry health family welfa...,delhi india march union minister narendra sing...,delhi india march bharat biotech serum institu...
1,india reported lowest number daily coronavirus...,india tally covid case rose with infection whi...,india coronavirus covid tally case rose with i...,india covid tally rose with over fresh case be...,india covid case rose with infection being rep...,daily rise coronavirus infection india recorde...,with coronavirus case being reported india cov...,india registered coronavirus infection lowest ...,india covid tally surpassed crore march with i...,daily covid case india registered increase fou...
2,vaccination based vulnerability infection prof...,mumbai maharashtra india march maharashtra hom...,prime minister narendra modi said monday that ...,anil deshmukh said order state better than wha...,delhi india february with provide better healt...,mumbai maharashtra india march maharashtra lea...,delhi high court march asked centre explain ra...,union home ministry monday said crore been san...,questioning ruling whether wanted permanent st...,prime minister narendra modi friday said produ...
3,dehradun uttarakhand india march shortly after...,jammu jammu kashmir india february senior cong...,dehradun uttarakhand india march predicting ch...,delhi india february congress general secretar...,suspense loom over fate leadership uttarakhand...,siddharth sharmanew delhi india march congress...,chacko resignation come huge shock congress co...,delhi india february senior congress leader fo...,former union minister veteran congress leader ...,uttarakhand chief minister trivendra singh raw...
4,delhi india february total assembly constituen...,delhi india february election commission india...,kumar gauravnew delhi india march bhartiya jan...,kolkata west bengal india march west bengal ch...,indore madhya pradesh india february congress ...,kolkata west bengal india march congress senio...,kolkata west bengal india march trinamool cong...,kolkata west bengal india march west bengal ch...,delhi india march taking jibe west bengal chie...,delhi india february congress adhir ranjan cho...
5,delhi india march covid vaccination concluded ...,delhi india march prime minister narendra modi...,delhi india march second phase vaccination cov...,delhi india march prime minister narendra modi...,dehradun india march uttarakhand governor baby...,delhi india march several union minister polit...,delhi india march indian medical association m...,delhi india march delhi lieutenant governor an...,delhi india march union defence minister rajna...,jammu jammu kashmir india march jammu kashmir ...
6,delhi india february union home ministry exten...,urging centre make agri law prestige issue sen...,agriculture ministry postponed next round talk...,agitating farmer union february objected delhi...,ghazipur uttar pradesh india february bharatiy...,farmer leader rakesh tikait monday took union ...,farmer group said they plan intensify their pr...,second round meeting between police farmer uni...,shamli uttar pradesh india february amid ongoi...,meerut uttar pradesh india february extending ...
7,indore madhya pradesh india february indore be...,delhi high court march asked centre explain ra...,chief justice india sharad bobde tuesday disag...,andhra pradesh government orally stated suprem...,vaccination based vulnerability infection prof...,warning that laxity implementing stringent mea...,delhi india march delhi high court wednesday i...,with several state country continuing report i...,delhi india march supreme court wednesday will...,de